In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 75.7 MB/s eta 0:00:00


# Loading Data

In [ ]:
import pandas as pd

# Assuming the file is named 'population.xlsx'
population = pd.read_excel('population.xlsx')
# Modify the "Belediyeler" column to extract only the first word
population['Belediyeler'] = population['Belediyeler'].apply(lambda x: x.split()[0])
# Display the first few rows of the modified dataset
population.head()

,Belediyeler,2019 yılı nüfusları
0,Adalar,15238
1,Arnavutköy,282488
2,Ataşehir,425094
3,Avcılar,448882
4,Bağcılar,745125


In [ ]:
# prompt: Using dataframe population: create the sum of second column using pivot method or a basic method

# Calculate the sum of the "2019 yılı nüfuslar" column
total_population = population["2019 yılı nüfusları"].sum()

# Print the result
print(f"Total population in 2019: {total_population}")
print("Number of districts:",population.Belediyeler.nunique())

Total population in 2019: 15519267
Number of districts: 39


In [ ]:
import json
import pandas as pd

# Open the JSON file with 'utf-8-sig' encoding to handle BOM
with open('turkey-geo.json', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# If the JSON has a 'features' key or similar, extract the relevant part (adjust based on your file structure)
if isinstance(data, dict) and 'features' in data:
    data = data['features']

# Convert the JSON data to a DataFrame
df = pd.DataFrame(data)

# Function to split the "Coordinates" field into Latitude and Longitude for the province
def split_coordinates(coordinates):
    try:
        lat, lon = map(float, coordinates.split(', '))
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

# Apply the function to the 'Coordinates' column to extract latitude and longitude
df[['Province_Latitude', 'Province_Longitude']] = df['Coordinates'].apply(split_coordinates)

# Define a function to extract district names and coordinates (and split them into Latitude and Longitude)
def extract_district_info(districts):
    if isinstance(districts, list):
        return [(district['District'], *map(float, district['Coordinates'].split(', '))) for district in districts]
    return None

# Apply the function to the 'Districts' column to extract district names and their coordinates
df['District_Info'] = df['Districts'].apply(extract_district_info)

# Explode the 'District_Info' column into separate rows (if there are multiple districts)
df_exploded = df.explode('District_Info')

# Split 'District_Info' into District name, Latitude, and Longitude
df_exploded[['District', 'District_Latitude', 'District_Longitude']] = pd.DataFrame(df_exploded['District_Info'].tolist(), index=df_exploded.index)

# Drop the original 'Districts', 'Coordinates', and 'District_Info' columns as they are no longer needed
df_exploded.drop(columns=['Districts', 'Coordinates', 'District_Info'], inplace=True)

# Display the resulting DataFrame
df_exploded.head(2)

,Province,PlateNumber,Province_Latitude,Province_Longitude,District,District_Latitude,District_Longitude
0,Adana,1,37.0,35.325,Aladağ,37.5485,35.3960
0,Adana,1,37.0,35.325,Ceyhan,37.0247,35.8175


In [ ]:
istanbul_df = df_exploded[df_exploded['Province'] == 'İstanbul']
istanbul_df.head()
print("Number of districts:",istanbul_df.District.nunique())

Number of districts: 39


In [ ]:
# prompt: I want you to join df and istanbul_df on df.Belediyeler and istanbul_df.District. Make it a inner join. Only add the second column of df to the end of join.

merged_df = pd.merge(istanbul_df, population, left_on='District', right_on='Belediyeler', how='inner')
merged_df.drop(columns=['Belediyeler'], inplace=True)
merged_df.rename(columns={'2019 yılı nüfusları': 'Population'}, inplace=True)
merged_df.drop(columns=['Province', 'PlateNumber','Province_Latitude','Province_Longitude'], inplace=True)
merged_df.head()

,District,District_Latitude,District_Longitude,Population
0,Adalar,40.867800,29.133100,15238
1,Arnavutköy,41.248259,28.681450,282488
2,Ataşehir,40.984749,29.106720,425094
3,Avcılar,40.980135,28.717547,448882
4,Bağcılar,41.044729,28.833714,745125


In [ ]:
merged_df.sort_values(by='Population', ascending=False).head()

,District,District_Latitude,District_Longitude,Population
17,Esenyurt,41.048920,28.658117,954579
25,Küçükçekmece,40.996000,28.774800,792821
4,Bağcılar,41.044729,28.833714,745125
27,Pendik,40.877500,29.272500,711894
36,Ümraniye,41.027219,29.127459,710280


In [ ]:
import numpy as np

# Given parameters
vehicle_ownership_rate = 0.25    # 25%
ev_penetration_rate = 0.02      # 2%
market_share = 0.3              # 20%
vehicles_per_station_capacity = 20  # One unit serves 20 cars

# Compute the number of EVs in each district
merged_df['Number_of_EVs'] = merged_df['Population'] * vehicle_ownership_rate * ev_penetration_rate

# Compute the number of EVs using our customer's stations
merged_df['EVs_using_customer_stations'] = merged_df['Number_of_EVs'] * market_share

# Compute the demand (number of capacity units needed) in each district
merged_df['Demand'] = merged_df['EVs_using_customer_stations'] / vehicles_per_station_capacity

# Round up the demand to the nearest integer, ensuring at least one unit of demand
merged_df['Demand'] = merged_df['Demand'].apply(lambda x: max(int(np.ceil(x)), 1))
merged_df=merged_df.drop(columns=["Number_of_EVs","EVs_using_customer_stations"])
# Display the DataFrame with the new normalized column
merged_df.head(5)


,District,District_Latitude,District_Longitude,Population,Demand
0,Adalar,40.867800,29.133100,15238,2
1,Arnavutköy,41.248259,28.681450,282488,22
2,Ataşehir,40.984749,29.106720,425094,32
3,Avcılar,40.980135,28.717547,448882,34
4,Bağcılar,41.044729,28.833714,745125,56


In [ ]:
stations=pd.read_excel('borusan.xlsx')

In [ ]:
stations=stations.iloc[:,3:9].drop(columns=['Nüfus','Mahalle'])
stations.columns=['Street','District','latitude','longitude']
stations.District=stations.District.str.replace('İ','I')
stations = stations[~stations['District'].isin(['ADALAR', 'ŞILE'])]
np.random.seed(42)
stations['Capacity'] = np.random.randint(5, 20, size=len(stations))
stations.head()

,Street,District,latitude,longitude,Capacity
0,BAĞCILAR100. YIL,BAĞCILAR,41.063656,28.852570,11
1,ESENLER15 TEMMUZ,ESENLER,41.062042,28.877897,8
2,BÜYÜKÇEKMECE19 MAYIS,BÜYÜKÇEKMECE,41.022483,28.581827,17
3,KADIKÖY19 MAYIS,KADIKÖY,40.977008,29.089437,19
4,ŞIŞLI19 MAYIS,ŞIŞLI,41.059865,28.991444,15


In [ ]:
stations.District.str.replace('İ','I').unique()

array(['BAĞCILAR', 'ESENLER', 'BÜYÜKÇEKMECE', 'KADIKÖY', 'ŞIŞLI',
       'EYÜPSULTAN', 'SULTANGAZI', 'BEŞIKTAŞ', 'GÜNGÖREN', 'SANCAKTEPE',
       'SULTANBEYLI', 'BEYKOZ', 'ÜSKÜDAR', 'ÜMRANIYE', 'BEYLIKDÜZÜ',
       'ARNAVUTKÖY', 'PENDIK', 'ÇATALCA', 'ESENYURT', 'TUZLA', 'SILIVRI',
       'FATIH', 'ÇEKMEKÖY', 'MALTEPE', 'BAŞAKŞEHIR', 'BAYRAMPAŞA',
       'AVCILAR', 'BEYOĞLU', 'ATAŞEHIR', 'KÜÇÜKÇEKMECE', 'BAKIRKÖY',
       'KARTAL', 'SARIYER', 'GAZIOSMANPAŞA', 'BAHÇELIEVLER',
       'ZEYTINBURNU', 'KAĞITHANE'], dtype=object)

In [ ]:
merged_df.District.str.upper().unique()

array(['ADALAR', 'ARNAVUTKÖY', 'ATAŞEHIR', 'AVCILAR', 'BAĞCILAR',
       'BAHÇELIEVLER', 'BAKIRKÖY', 'BAŞAKŞEHIR', 'BAYRAMPAŞA', 'BEŞIKTAŞ',
       'BEYKOZ', 'BEYLIKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE', 'ÇATALCA',
       'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATIH',
       'GAZIOSMANPAŞA', 'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL',
       'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDIK', 'SANCAKTEPE', 'SARIYER',
       'SILIVRI', 'SULTANBEYLI', 'SULTANGAZI', 'ŞILE', 'ŞIŞLI', 'TUZLA',
       'ÜMRANIYE', 'ÜSKÜDAR', 'ZEYTINBURNU'], dtype=object)

In [ ]:
merged_df[['District','Demand']].head()

,District,Demand
0,Adalar,2
1,Arnavutköy,22
2,Ataşehir,32
3,Avcılar,34
4,Bağcılar,56


In [ ]:
merged_df.Demand.sort_values(ascending=False)

,Demand
17,72
25,60
4,56
27,54
36,54
5,46
32,41
37,40
26,39
22,37


In [ ]:
import pulp
import numpy as np
import pandas as pd
from geopy.distance import geodesic

# Load district data
district_data=merged_df[merged_df.Demand>5].reset_index(drop=True)

# Extract district locations and demands
district_locations = district_data[['District_Latitude', 'District_Longitude']].values
district_demands = district_data['Demand'].values.astype(int)  # Ensure demands are integers
district_indices = district_data.index.values

# Generate 50 station locations evenly distributed
num_stations = len(stations)
#num_selected_stations = 50  # Increased from 25 to 30

# Generate stations
np.random.seed(42)  # For reproducibility
station_lats = stations.latitude
station_lons = stations.longitude
station_locations = list(zip(station_lats, station_lons))

# Assign capacities randomly between 5 and 30 (increase minimum capacity)
N_i = stations.Capacity.values


# Assign fixed costs
V = 100  # Variable cost per slot

# Fixed costs based on location
station_fixed_costs = []
for lat, lon in station_locations:
    distances = [geodesic((lat, lon), (d_lat, d_lon)).km for d_lat, d_lon in district_locations]
    nearest_district_idx = np.argmin(distances)
    population = district_data.loc[nearest_district_idx, 'Population']
    fixed_cost = population * 0.05  # Adjust coefficient as needed
    station_fixed_costs.append(fixed_cost)

# Total cost per station
station_total_costs = [station_fixed_costs[i] + V * N_i[i] for i in range(num_stations)]

# Distance threshold
theta = 5  # in kilometers

# Utilization rates
U_min = 0.1
U_max = 1.0

# Initialize the optimization problem
prob = pulp.LpProblem("EV_Charging_Stations_Optimization", pulp.LpMinimize)

# Decision variables
x = [pulp.LpVariable(f"x_{i}", cat='Binary') for i in range(num_stations)]
y = [[pulp.LpVariable(f"y_{i}_{j}", lowBound=0, cat='Integer')
    for j in range(len(district_data))]
    for i in range(num_stations)]

# Objective function
TotalCost = pulp.lpSum([station_total_costs[i] * x[i] for i in range(num_stations)])
prob += TotalCost

# Constraints

# 1. Station selection constraint
#prob += pulp.lpSum([x[i] for i in range(num_stations)]) == num_selected_stations, "Total_Stations_Selected"

# 2. Demand satisfaction constraints
for j in range(len(district_data)):
    prob += pulp.lpSum([y[i][j] for i in range(num_stations)]) == district_demands[j], f"Demand_Satisfaction_{j}"

# 3. Supply constraints
for i in range(num_stations):
    prob += pulp.lpSum([y[i][j] for j in range(len(district_data))]) <= N_i[i] * x[i], f"Supply_Limit_{i}"

# 4. Service distance constraint
for i in range(num_stations):
    for j in range(len(district_data)):
        distance = geodesic(station_locations[i], district_locations[j]).km
        if distance <= theta:
            s_ij = 1
        else:
            s_ij = 0
        # If s_ij is 0, y[i][j] must be 0
        prob += y[i][j] <= district_demands[j] * s_ij, f"Service_Distance_{i}_{j}"

# 5. Utilization balance constraints
for i in range(num_stations):
    min_utilization = int(U_min * N_i[i])
    max_utilization = int(U_max * N_i[i])
    prob += pulp.lpSum([y[i][j] for j in range(len(district_data))]) >= min_utilization * x[i], f"Min_Utilization_{i}"
    prob += pulp.lpSum([y[i][j] for j in range(len(district_data))]) <= max_utilization * x[i], f"Max_Utilization_{i}"

# Solve the problem
print('Started solving')

prob.solve(pulp.PULP_CBC_CMD(msg=True, timeLimit=600))

# Check solver status
status = pulp.LpStatus[prob.status]
print("Status:", status)

print(f"Optimal Total Cost: ${pulp.value(prob.objective):,.0f}\n")
selected_stations = []
for i in range(num_stations):
    if x[i].varValue == 1:
        selected_stations.append(i)
        station_capacity = N_i[i]
        total_supply = sum([y[i][j].varValue for j in range(len(district_data))])
        print(f"Station {i}:")
        print(f"  Location: {station_locations[i]}")
        print(f"  Capacity (Number of Slots): {station_capacity}")
        print(f"  Fixed Cost: {station_fixed_costs[i]:.2f}")
        print(f"  Variable Cost: {V * station_capacity}")
        print(f"  Total Cost: {station_total_costs[i]:.2f}")
        print(f"  Total Supply: {int(total_supply)}")
        if total_supply > station_capacity:
            print(f"  Warning: Total supply exceeds capacity!")
        supplied_districts = []
        for j in range(len(district_data)):
            supply = y[i][j].varValue
            if supply > 0:
                supplied_districts.append((district_data.loc[j, 'District'], int(supply)))
        print(f"  Supplies to Districts: {supplied_districts}\n")
# Verify that all districts' demands are met
for j in range(len(district_data)):
    total_supply = sum([y[i][j].varValue for i in range(num_stations)])
    print(f"District {district_data.loc[j, 'District']} Demand: {district_demands[j]}, Supplied: {int(total_supply)}")

Started solving
Status: Optimal
Optimal Total Cost: $1,363,639

Station 3:
  Location: (40.977008, 29.0894372)
  Capacity (Number of Slots): 19
  Fixed Cost: 21254.70
  Variable Cost: 1900
  Total Cost: 23154.70
  Total Supply: 19
  Supplies to Districts: [('Kadıköy', 19)]

Station 18:
  Location: (41.2132529, 28.7039075)
  Capacity (Number of Slots): 12
  Fixed Cost: 14124.40
  Variable Cost: 1200
  Total Cost: 15324.40
  Total Supply: 11
  Supplies to Districts: [('Arnavutköy', 11)]

Station 32:
  Location: (40.9670161, 29.2303824)
  Capacity (Number of Slots): 14
  Fixed Cost: 16801.05
  Variable Cost: 1400
  Total Cost: 18201.05
  Total Supply: 14
  Supplies to Districts: [('Sancaktepe', 14)]

Station 34:
  Location: (41.0172962, 28.6944236)
  Capacity (Number of Slots): 17
  Fixed Cost: 22444.10
  Variable Cost: 1700
  Total Cost: 24144.10
  Total Supply: 17
  Supplies to Districts: [('Avcılar', 1), ('Esenyurt', 16)]

Station 48:
  Location: (40.951875, 29.1016652)
  Capacity (Num

In [ ]:
# Filter the stations DataFrame based on selected_stations list
stations_filtered = stations[stations.index.isin(selected_stations)].reset_index(drop=True)

# Export the filtered DataFrame to a CSV file
stations_filtered.to_excel('filtered_stations.xlsx', index=False)

In [ ]:
stations[stations.index.isin(selected_stations)]

,Street,District,latitude,longitude,Capacity
3,KADIKÖY19 MAYIS,KADIKÖY,40.977008,29.089437,19
44,FATIHALI KUŞÇU,FATIH,41.020696,28.948447,15
47,EYÜPSULTANALIBEYKÖY,EYÜPSULTAN,41.077257,28.946893,16
65,BEŞIKTAŞARNAVUTKÖY,BEŞIKTAŞ,41.067177,29.043271,11
74,BAKIRKÖYATAKÖY 2. 5. 6. KISIM,BAKIRKÖY,40.977617,28.857788,14
80,BÜYÜKÇEKMECEATATÜRK,BÜYÜKÇEKMECE,41.012264,28.600402,16
96,BAĞCILARBAĞLAR,BAĞCILAR,41.022887,28.824829,10
105,PENDIKBAHÇELIEVLER,PENDIK,40.883911,29.235379,16
117,PENDIKBALLICA,PENDIK,40.990429,29.401439,16
128,ÇATALCABAŞAK,ÇATALCA,41.332951,28.445322,13
